In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,82.54,71,24,12.82,few clouds
1,1,Poum,NC,-20.2333,164.0167,78.89,77,5,12.97,clear sky
2,2,Mataura,NZ,-46.1927,168.8643,47.48,95,100,7.49,light rain
3,3,Quatre Cocos,MU,-20.2078,57.7625,75.09,63,26,15.37,scattered clouds
4,4,Georgetown,MY,5.4112,100.3354,87.73,70,20,6.91,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Mataura,NZ,-46.1927,168.8643,47.48,95,100,7.49,light rain
6,6,Arraial Do Cabo,BR,-22.9661,-42.0278,67.75,77,7,9.95,clear sky
8,8,Ushuaia,AR,-54.8000,-68.3000,46.06,49,75,11.50,broken clouds
11,11,Los Llanos De Aridane,ES,28.6585,-17.9182,64.15,84,0,4.00,clear sky
16,16,Castro,BR,-24.7911,-50.0119,50.43,90,9,1.95,clear sky
19,19,Busselton,AU,-33.6500,115.3333,66.56,71,47,16.35,scattered clouds
20,20,Olafsvik,IS,64.8945,-23.7142,48.06,70,93,13.73,overcast clouds
21,21,Punta Arenas,CL,-53.1500,-70.9167,42.91,56,75,20.71,broken clouds
22,22,Camacha,PT,33.0833,-16.3333,63.82,72,40,9.22,scattered clouds
23,23,Broken Hill,AU,-31.9500,141.4333,60.91,51,12,10.22,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.count())

City_ID                254
City                   254
Country                252
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Current Description    254
dtype: int64


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                252
City                   252
Country                252
Lat                    252
Lng                    252
Max Temp               252
Humidity               252
Cloudiness             252
Wind Speed             252
Current Description    252
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Mataura,NZ,47.48,light rain,-46.1927,168.8643,NaN
6,Arraial Do Cabo,BR,67.75,clear sky,-22.9661,-42.0278,NaN
8,Ushuaia,AR,46.06,broken clouds,-54.8000,-68.3000,NaN
11,Los Llanos De Aridane,ES,64.15,clear sky,28.6585,-17.9182,NaN
16,Castro,BR,50.43,clear sky,-24.7911,-50.0119,NaN
19,Busselton,AU,66.56,scattered clouds,-33.6500,115.3333,NaN
20,Olafsvik,IS,48.06,overcast clouds,64.8945,-23.7142,NaN
21,Punta Arenas,CL,42.91,broken clouds,-53.1500,-70.9167,NaN
22,Camacha,PT,63.82,scattered clouds,33.0833,-16.3333,NaN
23,Broken Hill,AU,60.91,few clouds,-31.9500,141.4333,NaN


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.count()

City                   252
Country                252
Max Temp               252
Current Description    252
Lat                    252
Lng                    252
Hotel Name             227
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
#Reset display options for dataframe just to make sure that all rows are populated
pd.set_option("display.max_rows", None, "display.max_columns", None)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Mataura,NZ,47.48,light rain,-46.1927,168.8643,Ellie's Villa
6,Arraial Do Cabo,BR,67.75,clear sky,-22.9661,-42.0278,Pousada Porto Praia
8,Ushuaia,AR,46.06,broken clouds,-54.8000,-68.3000,Albatross Hotel
11,Los Llanos De Aridane,ES,64.15,clear sky,28.6585,-17.9182,Valle Aridane
16,Castro,BR,50.43,clear sky,-24.7911,-50.0119,CHACARA BAILLY
19,Busselton,AU,66.56,scattered clouds,-33.6500,115.3333,Observatory Guest House
20,Olafsvik,IS,48.06,overcast clouds,64.8945,-23.7142,Hotel Olafsvik
21,Punta Arenas,CL,42.91,broken clouds,-53.1500,-70.9167,Hotel Hain
22,Camacha,PT,63.82,scattered clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
23,Broken Hill,AU,60.91,few clouds,-31.9500,141.4333,Royal Exchange Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Mataura,NZ,47.48,light rain,-46.1927,168.8643,Ellie's Villa
6,Arraial Do Cabo,BR,67.75,clear sky,-22.9661,-42.0278,Pousada Porto Praia
8,Ushuaia,AR,46.06,broken clouds,-54.8000,-68.3000,Albatross Hotel
11,Los Llanos De Aridane,ES,64.15,clear sky,28.6585,-17.9182,Valle Aridane
16,Castro,BR,50.43,clear sky,-24.7911,-50.0119,CHACARA BAILLY


In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))